This work book is going to handle all of the data that we obtained from wikidata which is the following

- Actor Data and names
- Film Actor Data and names
- Television Actor Data and names
- Voice Actor Data and names
- Director Data and names
- Film Director Data and names
- Screenwriter Data and names

The data collected is as follows

- Name
- Gender
- Date of Birth
- Height
- Place of birth
- Ethnic group
- Citizenship
- Eye colour
- Hair colour

First step lets read all of the csv's with data and begin to group them together

In [308]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

In [309]:
actor = pd.read_csv('./Wikidata_queries/data_full/actor_data_full.csv')
film_actor = pd.read_csv('./Wikidata_queries/data_full/film_actor_data_full.csv')
tv_actor = pd.read_csv('./Wikidata_queries/data_full/television_actor_data_full.csv')
voice_actor = pd.read_csv('./Wikidata_queries/data_full/voice_actor_data_full.csv')
director = pd.read_csv('./Wikidata_queries/data_full/director_data_full.csv')
film_director = pd.read_csv('./Wikidata_queries/data_full/film_director_data_full.csv')
swriter = pd.read_csv('./Wikidata_queries/data_full/screenwriter_data_full.csv')

C:\Users\aliwa\AppData\Local\Temp\ipykernel_12284\3059737156.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  actor = pd.read_csv('./Wikidata_queries/data_full/actor_data_full.csv')
C:\Users\aliwa\AppData\Local\Temp\ipykernel_12284\3059737156.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  swriter = pd.read_csv('./Wikidata_queries/data_full/screenwriter_data_full.csv')


Let's start by combining all of the actor datasets together and the director/screenwriter datasets together. There are likely to be duplicates and if we find those we can drop them.

In [310]:
print(actor.columns[0])
print(tv_actor.columns[0])
print(voice_actor.columns[0])
print(film_actor.columns[0])
print(film_director.columns[0])
print(director.columns[0])
print(swriter.columns[0])


actor
television_actor
voice_actor
film_actor
film_director
director
screenwriter


In [311]:
#change the first column name so that they are all the same
film_actor.rename(columns = {'film_actor':'actor'}, inplace=True)
tv_actor.rename(columns = {'television_actor':'actor'}, inplace=True)
voice_actor.rename(columns = {'voice_actor':'actor'}, inplace=True)
film_director.rename(columns = {'film_director':'dir_swriter'}, inplace=True)
director.rename(columns = {'director':'dir_swriter'}, inplace=True)
swriter.rename(columns = {'screenwriter':'dir_swriter'}, inplace=True)

In [312]:


actor_datasets = [actor,film_actor,tv_actor,voice_actor]
director_datasets = [director,film_director,swriter]

all_actors = pd.concat(actor_datasets)
all_dir = pd.concat(director_datasets)

#we are going to drop place of eye colour, hair colour, birth place, and citizenship data because we are likely not going to use it for our purposes
all_actors.drop('place_of_birth', inplace=True, axis=1)
all_actors.drop('citizenship', inplace=True, axis=1)
all_actors.drop('eye_colour', inplace=True, axis=1)
all_actors.drop('hair_colour', inplace=True, axis=1)

all_dir.drop('place_of_birth', inplace=True, axis=1)
all_dir.drop('citizenship', inplace=True, axis=1)
all_dir.drop('eye_colour', inplace=True, axis=1)
all_dir.drop('hair_colour', inplace=True, axis=1)

In [313]:
print(all_actors.columns)
print(all_dir.columns)

Index(['actor', 'gender', 'dob', 'height', 'ethnic_group'], dtype='object')
Index(['dir_swriter', 'gender', 'dob', 'height', 'ethnic_group'], dtype='object')


now let's check how many duplicates there are in each dataframe

In [314]:
print(f'The total amount of actor rows are {len(all_actors)}, the number of unique values are {all_actors.actor.nunique()}')
print(f'The total amount of actor rows are {len(all_dir)}, the number of unique values are {all_dir.dir_swriter.nunique()}')

The total amount of actor rows are 763194, the number of unique values are 395291
The total amount of actor rows are 394761, the number of unique values are 179655


First we will jut drop all of the rows that are completely identical.

In [315]:
all_actors.drop_duplicates(keep='first', inplace=True)
all_dir.drop_duplicates(keep='first', inplace=True)
print(f'The total amount of actor rows left are {len(all_actors)}, but the number of unique actors is still {all_actors.actor.nunique()}')
print(f'The total amount of actor rows left are {len(all_dir)}, but the number of unique directors is still {all_dir.dir_swriter.nunique()}')

The total amount of actor rows left are 403549, but the number of unique actors is still 395291
The total amount of actor rows left are 182884, but the number of unique directors is still 179655


We can see that we have made a lot of progress, but there is still data that needs to be cleaned so that we can have only one actor/director per row. To start we can start by grouping the objects by the actor wikidata page and then looking at the ones that have more than one value to understand what the differences are

In [316]:
all_actors.sort_values('actor',inplace=True)
all_dir.sort_values('dir_swriter', inplace=True)
dup_actors = all_actors[all_actors.actor.duplicated(keep=False)]
dup_dirs = all_dir[all_dir.dir_swriter.duplicated(keep=False)]
grouped_dup_actors = dup_actors.groupby('actor')
grouped_dup_dirs = dup_dirs.groupby('dir_swriter')
print(all_actors.columns)
print(all_dir.columns)

Index(['actor', 'gender', 'dob', 'height', 'ethnic_group'], dtype='object')
Index(['dir_swriter', 'gender', 'dob', 'height', 'ethnic_group'], dtype='object')


So let's clear up any issues arising from two different rows just not having the 

In [317]:
counterg = []
counterd = []
counterh = []
countere = []
for name, group in grouped_dup_actors:
        gend = []
        dob = [] 
        height = []
        ethnicity = []

        cols = [['gender',gend],['dob',dob],['height',height],['ethnic_group',ethnicity]]

        for row in group.iterrows():
                for col,lst in cols:
                        if not pd.isna(row[1][col]):
                                lst.append(row[1][col])
        counterg.append(len(cols[0][1]))
        counterd.append(len(cols[1][1]))
        counterh.append(len(cols[2][1]))
        countere.append(len(cols[3][1]))
        for col, lst in cols:
                if len(lst) == 1:
                        print('we got one a')
                        row[col] = lst[0]

for name, group in grouped_dup_dirs:
        gend = []
        dob = [] 
        height = []
        ethnicity = []

        cols = [['gender',gend],['dob',dob],['height',height],['ethnic_group',ethnicity]]

        for row in group.iterrows():
                for col,lst in cols:
                        if not pd.isna(row[1][col]):
                                lst.append(row[1][col])
        for col, lst in cols:
                if len(lst) == 1:
                        print('we got one b')
                        row[1][col] = lst[0]



This means that there are no values in which one row has NaN and the other has a value in the actor's dataframe or in the directors data frame which means that we are dealing with actual duplicates which have conflicting data, to move forward with this without spending more time I am going to drop the duplicate data

In [318]:
print(f'The total amount of actor data is {all_actors.actor.nunique()} and the amount of data to drop is {len(grouped_dup_actors)}, which is {100*len(grouped_dup_actors)/all_actors.actor.nunique()}%')
print(f'The total amount of director data is {len(all_dir)} and the amount of data to drop is {len(grouped_dup_dirs)}, which is {100*len(grouped_dup_dirs)/all_dir.dir_swriter.nunique()}%')

The total amount of actor data is 395291 and the amount of data to drop is 7904, which is 1.9995395797020423%
The total amount of director data is 182884 and the amount of data to drop is 3054, which is 1.6999248559739502%


Before we drop the data we can run some cleaning on the gender data since we only want to keep either Male or Female. We are also going to keep the NaN values because we have already verified that they are not conflicts with NaN values and we can still use that data to maybe verify other data

In [319]:
all_actors_clean = all_actors.copy()
all_dir_clean = all_dir.copy()

male = 'http://www.wikidata.org/entity/Q6581097' 
female =  'http://www.wikidata.org/entity/Q6581072'

all_actors_clean.drop(
    all_actors_clean.loc[
        (all_actors_clean['gender']!=male) & 
        (all_actors_clean['gender']!=female) &
        (all_actors_clean.gender.notna())
        ].index, inplace=True)

all_dir_clean.drop(
    all_dir_clean.loc[
        (all_dir_clean['gender']!=male) & 
        (all_dir_clean['gender']!=female) &
        (all_dir_clean.gender.notna())
        ].index, inplace=True)

print(f'Before there was {all_actors.actor.nunique()} unique actor data and now there is {all_actors_clean.actor.nunique()}')
print(f'Before there was {all_dir.dir_swriter.nunique()} unique director data and now there is {all_dir_clean.dir_swriter.nunique()}')
print(f'The length of the actor list is now {len(all_actors_clean)}')
print(f'The length of the director list is now {len(all_dir_clean)}')

Before there was 395291 unique actor data and now there is 394530
Before there was 179655 unique director data and now there is 179425
The length of the actor list is now 402736
The length of the director list is now 182623


Now we can drop the duplicates and start to join the naming lists

In [320]:
all_actors_clean.drop(all_actors_clean.loc[all_actors_clean.actor.duplicated(keep=False)].index, inplace=True)
all_dir_clean.drop(all_dir_clean.loc[all_dir_clean.dir_swriter.duplicated(keep=False)].index, inplace=True)

print(f'The new length of the actor list is {len(all_actors_clean)}')
print(f'The new length of the directors list is {len(all_dir_clean)}')

The new length of the actor list is 386668
The new length of the directors list is 174323


Let's double check that we now only have unique actors

In [321]:
print(f'The number of unqiue actor names are {all_actors_clean.actor.nunique()}')
print(f'The number of unqiue director names are {all_dir_clean.dir_swriter.nunique()}')

The number of unqiue actor names are 386668
The number of unqiue director names are 174323


Now that we have dealt with that we can go ahead and gather the naming data for everything and then start to make joins

In [322]:
actor_name_F = pd.read_csv('./Wikidata_queries/naming_lists/actor_name_female.csv')
actor_name_M = pd.read_csv('./Wikidata_queries/naming_lists/actor_name_male.csv')
film_actor_name = pd.read_csv('./Wikidata_queries/naming_lists/film_actor_names.csv')
tv_actor_name = pd.read_csv('./Wikidata_queries/naming_lists/television_actor_names.csv')
voice_actor_name = pd.read_csv('./Wikidata_queries/naming_lists/voice_actor_names.csv')

dir_name = pd.read_csv('./Wikidata_queries/naming_lists/director_names.csv')
film_dir_name = pd.read_csv('./Wikidata_queries/naming_lists/film_director_names.csv')
swriter_name = pd.read_csv('./Wikidata_queries/naming_lists/screenwriter_names.csv')

gender_name = pd.read_csv('./Wikidata_queries/naming_lists/gender_names.csv')
ethn_name = pd.read_csv('./Wikidata_queries/naming_lists/ethnic_group_names.csv')

We are going to put the data together right away and drop the duplicates and then check if there are any wikidata ID's that have several names attached which should not happen

In [323]:
act_csvs = [actor_name_M, actor_name_F, film_actor_name, tv_actor_name, voice_actor_name]
dir_csvs = [dir_name, film_dir_name, swriter_name]

actor_name_map = pd.concat(act_csvs)
dir_name_map = pd.concat(dir_csvs)

print(actor_name_map.columns)
print(dir_name_map.columns)

Index(['item', 'itemLabel'], dtype='object')
Index(['item', 'itemLabel'], dtype='object')


In [324]:
actor_name_map.drop_duplicates(keep='first', inplace=True)
dir_name_map.drop_duplicates(keep='first', inplace=True)

print(f'The length of the actor name map is {len(actor_name_map)}, and the number of unique values is {actor_name_map.item.nunique()}')
print(f'The length of the director name map is {len(dir_name_map)}, and the number of unique values is {dir_name_map.item.nunique()}')

The length of the actor name map is 352555, and the number of unique values is 352555
The length of the director name map is 179643, and the number of unique values is 179643


Everything looks good from there and now let's do the same for the ethnicity name map and the gender name map

In [325]:
ethn_name_map = ethn_name.copy()
gender_name_map = gender_name.copy()

ethn_name_map.drop_duplicates(keep='first', inplace=True)
gender_name_map.drop_duplicates(keep='first', inplace=True)

print(f'The length of the gender name map is {len(gender_name_map)}, and the number of unique values is {gender_name_map.item.nunique()}')
print(f'The length of the ethnicity name map is {len(ethn_name_map)}, and the number of unique values is {ethn_name_map.item.nunique()}')

The length of the gender name map is 1402, and the number of unique values is 1402
The length of the ethnicity name map is 16412, and the number of unique values is 16412


Now let's start to join all of the dataframe together

In [326]:
def replace_col (df, old_names, new_names):
    assert len(old_names) == len(new_names),'There lists are not of the same length'
    for i in range(len(old_names)):
        df.rename(columns={old_names[i]:new_names[i]}, inplace=True)


all_actors_temp1 = all_actors_clean.merge(actor_name_map, how='left', left_on='actor', right_on='item')
all_dir_temp1 = all_dir_clean.merge(dir_name_map, how='left', left_on='dir_swriter', right_on='item')
all_actors_temp1.drop('item', axis=1, inplace=True)
replace_col(all_actors_temp1, ['itemLabel'], ['name'])
all_dir_temp1.drop('item', axis=1, inplace=True)
replace_col(all_dir_temp1, ['itemLabel'], ['name'])


all_actors_temp2 = all_actors_temp1.merge(ethn_name_map, how='left', left_on='ethnic_group', right_on='item')
all_dir_temp2 = all_dir_temp1.merge(ethn_name_map, how='left', left_on='ethnic_group', right_on='item')
all_actors_temp2.drop('item', axis=1, inplace=True)
replace_col(all_actors_temp2, ['itemLabel'], ['ethnicity'])
all_dir_temp2.drop('item', axis=1, inplace=True)
replace_col(all_dir_temp2, ['itemLabel'], ['ethnicity'])


replace_col(all_actors_temp2,['gender'], ['gender_wikiID'])
replace_col(all_dir_temp2,['gender'], ['gender_wikiID'])

all_actors_temp3 = all_actors_temp2.merge(gender_name_map, how='left', left_on='gender_wikiID', right_on='item')
all_dir_temp3 = all_dir_temp2.merge(gender_name_map, how='left', left_on='gender_wikiID', right_on='item')
all_actors_temp3.drop('item', axis=1, inplace=True)
replace_col(all_actors_temp3, ['itemLabel'], ['gender'])
all_dir_temp3.drop('item', axis=1, inplace=True)
replace_col(all_dir_temp3, ['itemLabel'], ['gender'])

print(all_actors_temp3.columns)
print(all_dir_temp3.columns)

Index(['actor', 'gender_wikiID', 'dob', 'height', 'ethnic_group', 'name',
       'ethnicity', 'gender'],
      dtype='object')
Index(['dir_swriter', 'gender_wikiID', 'dob', 'height', 'ethnic_group', 'name',
       'ethnicity', 'gender'],
      dtype='object')


Now we can drop the columns that are not useful and rearrange them to get the final Dataframe

In [327]:
all_actors_temp3.drop('gender_wikiID', axis=1, inplace=True)
all_dir_temp3.drop('gender_wikiID', axis=1, inplace=True)
all_actors_temp3.drop('ethnic_group', axis=1, inplace=True)
all_dir_temp3.drop('ethnic_group', axis=1, inplace=True)
replace_col(all_actors_temp3, ['actor', 'dob'], ['wikiDataID', 'birth'])
replace_col(all_dir_temp3, ['dir_swriter', 'dob'], ['wikiDataID', 'birth'])
print(all_actors_temp3.columns)
print(all_dir_temp3.columns)

Index(['wikiDataID', 'birth', 'height', 'name', 'ethnicity', 'gender'], dtype='object')
Index(['wikiDataID', 'birth', 'height', 'name', 'ethnicity', 'gender'], dtype='object')


In [336]:
actors_final = pd.DataFrame()
dir_final = pd.DataFrame()

actors_final[[
    'name',
    'gender',
    'birth',
    'height',
    'ethnicity',
    'wikidata_id']] = all_actors_temp3[['name', 'gender', 'birth', 'height', 'ethnicity', 'wikiDataID']]

dir_final[[
    'name',
    'gender',
    'birth',
    'height',
    'ethnicity',
    'wikidata_id']] = all_dir_temp3[['name', 'gender', 'birth', 'height', 'ethnicity', 'wikiDataID']]



Now let's clean the birth dates as well as the wikidataIDs

In [337]:
actors_final.birth = actors_final[actors_final.birth.notna()].birth.apply(lambda x: x.split('T')[0])
dir_final.birth = dir_final[dir_final.birth.notna()].birth.apply(lambda x: x.split('T')[0])

actors_final.birth = actors_final[actors_final.birth.notna()].birth.apply(lambda x: np.nan if x.find('.org') != -1 else x)
dir_final.birth = dir_final[dir_final.birth.notna()].birth.apply(lambda x: np.nan if x.find('.org') != -1 else x)

It looks like there are fictional characters in here as well as actor's with birthdays that are not actual days

In [338]:
pd.to_datetime(actors_final[actors_final['birth'].notna()].birth, format=r"%Y-%m-%d", errors='coerce')
pd.to_datetime(dir_final[dir_final.birth.notna()].birth, format=r"%Y-%m-%d", errors='coerce')

actors_final

,name,gender,birth,height,ethnicity,wikidata_id
0,Henk Rigters,male,1915-10-06,NaN,NaN,http://www.wikidata.org/entity/Q100001260
1,Florian Jahr,male,1983-06-23,NaN,NaN,http://www.wikidata.org/entity/Q1000015
2,Ulrike Scheel,female,2000-01-01,NaN,NaN,http://www.wikidata.org/entity/Q1000089
3,Nadia Boule,female,1984-10-01,NaN,NaN,http://www.wikidata.org/entity/Q100011609
4,Renee Goddard,female,1923-01-01,NaN,NaN,http://www.wikidata.org/entity/Q1000204
...,...,...,...,...,...,...
386663,Eleonora Bolla,female,1986-12-22,NaN,NaN,http://www.wikidata.org/entity/Q99997567
386664,Lison Daniel,female,1992-06-12,NaN,NaN,http://www.wikidata.org/entity/Q99997711
386665,Giovanni Maria Buzzatti,male,NaN,NaN,NaN,http://www.wikidata.org/entity/Q99997849
386666,Gabriella Infelise,female,NaN,NaN,NaN,http://www.wikidata.org/entity/Q99998028


In [341]:
actors_final['wikidata_id'] = actors_final['wikidata_id'].apply(lambda x: x.split('/')[-1])
dir_final['wikidata_id'] = dir_final['wikidata_id'].apply(lambda x: x.split('/')[-1])

Okay some final numbers before we part ways with this data

In [347]:
actors_final.describe()

,name,gender,birth,height,ethnicity,wikidata_id
count,344814,378155,285949,16617.0,1255,386668
unique,338989,2,51436,436.0,260,386668
top,Nana,male,2000-01-01,160.0,African Americans,Q100001260
freq,5,211122,2508,964.0,297,1


In [348]:
dir_final.describe()

,name,gender,birth,height,ethnicity,wikidata_id
count,174307,164321,118734,1102.0,644,174323
unique,171503,2,35534,148.0,183,174323
top,Jan Novák,male,2000-01-01,170.0,African Americans,Q1000002
freq,5,128310,1739,62.0,155,1


Let's put this away for now and save it to a csv

In [349]:
actors_final.to_csv('./Wikidata_Actors.csv')
dir_final.to_csv('./Wikidata_Directors.csv')